In [2]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import time
import datetime
import praw
from praw.models import MoreComments
import regex
from urllib.error import HTTPError


reddit = praw.Reddit(client_id = '#####', 
                     client_secret = '#####', 
                     user_agent = 'SubScraper', 
                     username = 'KaranjotSinghV', 
                     password = '#####')

In [2]:
def getPushshiftData(after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [3]:
flairs = ["AskIndia", "Non-Political", "Scheduled", "Photography", "Science/Technology", "Politics", 
          "Business/Finance", "Policy/Economy", "Sports", "Food", "Coronavirus"]

In [4]:
post_ids = []

In [5]:
def isEnglish(string):
    try:
        string.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

### Function for collecting features of a post

In [6]:
def collectSubData(subm):
    
    subData = list() #list to store data points
    
    unique = subm['id']

    title = subm['title']
    
    url = subm['url']
    
    score = subm['score']
    
    commentsCount = subm['num_comments']

    timestamp = datetime.datetime.fromtimestamp(subm['created_utc'])
    
    try:
        body = subm['selftext']
    except:
        body = "NaN"
    
    try:
        flair = subm['link_flair_text']
    except:
        flair = "NaN"
    
    if flair in flairs and isEnglish(title) == True and isEnglish(body) == True:
        
        subData.append((unique, title, url, body, score, commentsCount, timestamp, flair))
        post_ids.append(unique)
        subStats[unique] = subData

Collecting posts posted between 16th February, 2020 and 24th April, 2020

In [7]:
#subreddit to query
sub = 'india'

#before and after dates
before = 1587686400 #24th April 2020
after = 1581814800  #16th Feb 2020

subCount = 0
subStats = {}

In [8]:
data = getPushshiftData(after, before, sub)

# will run until all posts have been gathered 
# from the 'after' date up until 'before' date

while len(data) > 0:
    
    for submission in data:
        
        collectSubData(submission)
                
    # Calls getPushshiftData() with the created date of the last submission
    
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(after, before, sub)
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1581814800&before=1587686400&subreddit=india
1000
2020-02-18 21:18:11
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1582040891&before=1587686400&subreddit=india
1000
2020-02-21 13:54:27
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1582273467&before=1587686400&subreddit=india
1000
2020-02-24 11:46:31
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1582524991&before=1587686400&subreddit=india
1000
2020-02-26 12:55:08
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1582701908&before=1587686400&subreddit=india
1000
2020-02-28 05:19:47
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1582847387&before=1587686400&subreddit=india
1000
2020-03-01 10:32:48
https://api.pushshift.io/reddit/search/submission/?&size=1000&after=1583038968&before=1587686400&subreddit=india
1000
2020-03-03 14:12:27
https://api.pushshift.io/reddit/se

In [9]:
print(str(len(subStats)) + " submissions have been added to list")

23877 submissions have added to list


### Retrieving Comments using PRAW

In [10]:
exNum = len(post_ids)
comments = []

In [11]:
def collectComments(unique):
    
    try:
        post = reddit.submission(unique) 

        post.comments.replace_more(limit=None)
    
        comms = ''    
        count = 0

        for top_level_comment in post.comments:

            comms = comms + ' ' + top_level_comment.body
            count += 1     

            if(count > 10):
                break
        
        comments.append(comms)
        
    except HTTPError as err:
        
        if err.code == 404:
            comments.append("NaN")
        else:
            raise

In [12]:
for count, ID in enumerate(post_ids):
    
    print("Collecting comments for post - " + str(count))
    collectComments(ID)

Concatenating Comments with other features!

In [35]:
subStats_list = []
for ind in range(len(post_ids)):
    
    substat = list(subStats[post_ids[ind]][0])
    subStats_list.append(substat)

In [36]:
for ind in range(len(subStats_list)):
    
    subStats_list[ind].insert(4, comments[ind])

In [40]:
subStats_list[5967]

['fgqdrj',
 'Interview on internet shutdowns',
 'https://netzpolitik.org/2020/jordan-throttles-not-blocks-internet-access-shutdowns-keepiton/',
 '',
 ' > netzpolitik.org: What is the most effective – that is repressive – type of a net block you encountered (not including the frequency in which blocks occur)?\n\n> Berhan Taye: The most pervasive internet shutdowns that have put many people in darkness is the shutdown in Kashmir. As the shutdown lasted for more than five months, journalists, human rights defenders, activists, and others found it hard to reach out to the world to tell the gruesome human rights violations that were happening in Jammu and Kashmir. Given the situation that was happening on the ground, this is probably the most pervasive shutdown we have seen in a while.',
 1,
 1,
 datetime.datetime(2020, 3, 11, 8, 30, 17),
 'Politics']

### Creating .csv

In [43]:
def updateSubs_file():
    
    upload_count = 0
    
    location = "/home/vilkhu/MIDAS 2020/Data/"
    
    filename = input()
    file = location + filename
    
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        
        a = csv.writer(file, delimiter=',')
        
        headers = ["Post ID", "Title", "URL", "Body","Comments", "Score", "Comments Count", "Publish Date", "Flair"]
        
        a.writerow(headers)
        
        for ind in range(len(subStats_list)):
            a.writerow(subStats_list[ind])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")

In [44]:
updateSubs_file()

PUSHSHIFT-Data.csv
23877 submissions have been uploaded


In [3]:
df = pd.read_csv("Data/PUSHSHIFT-Data.csv")
df.head()

,Post ID,Title,URL,Body,Comments,Score,Comments Count,Publish Date,Flair
0,f4j4os,Loudspeaker in temple too loud,https://www.reddit.com/r/india/comments/f4j4os...,Where can I complain against loudspeaker nuisa...,Call the police. If it doesn't work post anon...,1,45,2020-02-16 06:43:51,AskIndia
1,f4jmon,Life''s Calling !,https://www.reddit.com/r/india/comments/f4jmon...,"***Life''s Calling !*** \n\n***Dear Friends ,*...",Tldr? It's not loosing it's losing.. everythi...,1,9,2020-02-16 07:23:27,AskIndia
2,f4jpyp,"Another wall goes up for Trump, this time in I...",https://www.reuters.com/article/us-india-usa-t...,NaN,Deewaar wahin banayenge!,1,2,2020-02-16 07:30:38,Politics
3,f4jqqi,UP rickshaw puller invites Modi to his daughte...,https://www.theweek.in/news/india/2020/02/15/u...,NaN,PR kya hai ji :) I don't understand why peopl...,1,5,2020-02-16 07:32:23,Non-Political
4,f4jtws,Maharashtra Set To Roll Out NPR From May 1 As ...,https://news.abplive.com/news/india/maharashtr...,NaN,Uddhav chya aaichi gand!\n\nThey just changed...,1,6,2020-02-16 07:39:14,Politics


In [4]:
df["Flair"].value_counts()

Coronavirus           7597
Non-Political         5335
Politics              3798
AskIndia              3433
Science/Technology     882
Policy/Economy         771
Business/Finance       739
Photography            614
Food                   273
Sports                 236
Scheduled              199
Name: Flair, dtype: int64